<a href="https://colab.research.google.com/github/afshalliaquat/Sentimental_Analysis_NaiveBayes.ipynb/blob/main/Sentimental_Analysis_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.txt',sep = ';',header=None,names= ['text','emotions'])

In [3]:
unique_emotions = df['emotions'].unique()
emotion_no = {}
i=0

for emo in unique_emotions:
  emotion_no[emo]=i
  i+=1

df['emotions'] = df['emotions'].map(emotion_no)

In [4]:
df['text'] = df['text'].apply(lambda x:x.lower())

In [5]:
import string

In [6]:
def remove_punc(text):
  return text.translate(str.maketrans('','',string.punctuation))

In [7]:
df['text'] = df['text'].apply(remove_punc)

In [8]:
def remove_no(text):
  string = ''
  for i in text:
    if not i.isdigit():
      string += i
  return string


In [9]:
df['text'] = df['text'].apply(remove_no)

In [10]:
def remove_emoji(text):
  string = ''
  for i in text:
    if i.isascii():
      string+=i
  return string

In [11]:
df['text'] = df['text'].apply(remove_emoji)

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [13]:
nltk.download('stopwords')
nltk.download('punkt')       # for word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def remove(txt):
  words = txt.split()
  cleaned = []
  for i in words:
    if not i in stop_words:
      cleaned.append(i)
  return ' '.join(cleaned)

In [16]:
df['text'] = df['text'].apply(remove)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = df['text']
y= df['emotions']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer  #Bag of words (only 0 and 1s)

In [21]:
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

In [23]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [24]:
model = MultinomialNB()
model.fit(X_train_bow,y_train)
y_pred = model.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.739433117851815

In [25]:
Parameter_NB = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'fit_prior': [True, False]
}

In [26]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [27]:
grid = GridSearchCV(estimator=model,cv=5,param_grid=Parameter_NB)
grid.fit(X_train_bow,y_train)
new_model_NB =grid.best_estimator_
y_pre = new_model_NB.predict(X_test_bow)
accuracy_score(y_test,y_pre)

0.7896568871208354

In [28]:
model2 = MultinomialNB()
model2.fit(X_train_vector, y_train)
y_pred_tfidf = model2.predict(X_test_vector)
accuracy_score(y_test,y_pred_tfidf)

0.6429636996519145

In [31]:
grid = GridSearchCV(estimator=model2,cv=5,param_grid=Parameter_NB)
grid.fit(X_train_vector,y_train)
new_model_NB =grid.best_estimator_
y_pre_vector = new_model_NB.predict(X_test_vector)
accuracy_score(y_test,y_pre)

0.7916459472899056

In [33]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pre))
print(classification_report(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pre_vector))


              precision    recall  f1-score   support

           0       0.67      0.93      0.78       530
           1       0.88      0.61      0.72       275
           2       0.95      0.25      0.39       161
           3       1.00      0.09      0.16        78
           4       0.87      0.44      0.58       253
           5       0.74      0.93      0.82       714

    accuracy                           0.74      2011
   macro avg       0.85      0.54      0.58      2011
weighted avg       0.78      0.74      0.71      2011

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       530
           1       0.85      0.72      0.78       275
           2       0.77      0.45      0.57       161
           3       0.77      0.31      0.44        78
           4       0.84      0.65      0.73       253
           5       0.79      0.91      0.84       714
           6       0.00      0.00      0.00         0

    accuracy           